# Análisis de oferta

Solución para hacer análisis de oferta del modelo de abastecimiento estratégico.

In [3]:
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Paquetes de consulta a traves de Socrata

from sodapy import Socrata
identificador_columnas=pd.read_csv('datos/identificador_columnas.csv')
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
identificador_columnas=identificador_columnas[['Unificado','API_DA_S_I', 'API_DA_S_II']]
identificador_columnas=identificador_columnas.dropna()
identificador_columnas=identificador_columnas.set_index('Unificado')
identificador_columnas.drop('Fuente',axis=0,inplace=True)
identificador_datos={'SECOP I':'f789-7hwg','SECOP II':'jbjy-vk9h','TVEC':'rgxm-mmea'}
valor_columnas={'SECOP I':'API_DA_S_I','SECOP II':'API_DA_S_II','TVEC':'API_DA_S_TVEC'}
### Definición de cliente para Socrata

cliente=Socrata('www.datos.gov.co',None,timeout=1000000)


In [5]:
Queries_list=[]
results=[]
for i in ['SECOP I','SECOP II']:
    query_SECOP="""
    SELECT 
        {2} as entidad,
        {3} as departamento_entidad,
        {4} as orden,
        {5} as nit_entidad,
        count({0}) as cantidad,
        sum({1}) as suma_valor_contrato,
        date_extract_y({6}) as año
    Group by 
        {2},{3},{4},{5},{6}
    LIMIT 
        10000000
    """.format(identificador_columnas[valor_columnas[i]]['ID Contrato'],identificador_columnas[valor_columnas[i]]['Valor del contrato'],
    identificador_columnas[valor_columnas[i]]['Entidad'],identificador_columnas[valor_columnas[i]]['Departamento Entidad'],
    identificador_columnas[valor_columnas[i]]['Orden'],identificador_columnas[valor_columnas[i]]['NIT Entidad'],
    identificador_columnas[valor_columnas[i]]['Fecha de Firma'])
    Queries_list.append(query_SECOP)
    results_detalle = cliente.get(identificador_datos[i], query=query_SECOP)
    results.append(results_detalle)


In [6]:
results_dataframe=pd.concat([pd.DataFrame(results[0]),pd.DataFrame(results[1])])
results_dataframe.reset_index(inplace=True)
results_dataframe.drop('index',axis=1,inplace=True)
results_dataframe['suma_valor_contrato']=results_dataframe['suma_valor_contrato'].astype(float)
results_dataframe['cantidad']=results_dataframe['cantidad'].astype(int)
results_dataframe['orden']=results_dataframe['orden'].str.upper()
results_dataframe['departamento_entidad']=results_dataframe['departamento_entidad'].replace('Distrito Capital de Bogotá','Bogotá DC')\
    .replace('San Andrés Providencia y Santa Catalina','San Andrés, Providencia y Santa Catalina').replace('Norte De Santander','Norte de Santander')

In [7]:
results_dataframe_df=results_dataframe.groupby(['entidad','nit_entidad','año','orden','departamento_entidad']).agg({'cantidad':'sum','suma_valor_contrato':'sum'}).reset_index()

In [8]:
results_dataframe_df.to_csv('datos/contratos_entidad.csv',index=False)

In [13]:
dptos={}
for i in results_dataframe['departamento_entidad'].unique():
    dptos.update({i:[i]})
dptos['Bogotá DC']=['Bogotá DC','Distrito Capital de Bogotá']
dptos['San Andrés, Providencia y Santa Catalina']=['San Andrés, Providencia y Santa Catalina','San Andrés Providencia y Santa Catalina']
dptos['Norte de Santander']=['Norte de Santander','Norte De Santander']

In [15]:
import json
with open("datos/dptos.json", "w",encoding='utf-8') as write_file:
    json.dump(dptos, write_file, indent=4,ensure_ascii=False)